In [3]:
import pandas as pd

In [4]:
data_import = pd.read_csv('/Users/sitanshurupani/Data Science Coursework/Sem 4/GQP - National Parks/DataSpell Project/Data/Adams Data Cleaned/ALL_Years.csv')
weather_import = pd.read_csv('/Users/sitanshurupani/Data Science Coursework/Sem 4/GQP - National Parks/DataSpell Project/Data/Hourly_Weather_Master.csv')

In [13]:
data_import.dtypes

SrNo          int64
Month        object
Day           int64
1:00          int64
2:00          int64
3:00          int64
4:00          int64
5:00          int64
6:00          int64
7:00          int64
8:00          int64
9:00          int64
10:00         int64
11:00         int64
12:00         int64
13:00         int64
14:00         int64
15:00         int64
16:00         int64
17:00         int64
18:00         int64
19:00         int64
20:00         int64
21:00         int64
22:00         int64
23:00         int64
24:00:00      int64
Total       float64
DOW          object
Year          int64
dtype: object

In [16]:
cols=["Year","Month","Day"]
data_import['Full_Date'] = data_import[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
data_import['Full_Date'] = pd.to_datetime(data_import['Full_Date'])

In [58]:
weather_import['Full_Date'] = pd.to_datetime(weather_import['Date'])
weather_import['Hour'] = weather_import['dt_iso'].str[11:13]
weather_import['Hour'] = pd.to_datetime(weather_import['Hour'], format='%H').dt.hour

In [59]:
data_import.head()

,SrNo,Month,Day,1:00,2:00,3:00,4:00,5:00,6:00,7:00,...,19:00,20:00,21:00,22:00,23:00,24:00:00,Total,DOW,Year,Full_Date
0,0,MAY,1,0,1,0,0,11,2,2,...,9,6,1,1,0,0,208.0,Wed,2019,2019-05-01
1,1,MAY,2,0,0,0,0,1,3,0,...,21,13,0,4,0,1,217.0,Thurs,2019,2019-05-02
2,2,MAY,3,0,0,0,0,9,6,1,...,9,2,0,1,0,0,189.0,Fri,2019,2019-05-03
3,3,MAY,4,0,0,0,0,9,4,1,...,36,25,2,2,2,3,570.0,Sat,2019,2019-05-04
4,4,MAY,5,0,0,0,1,34,20,1,...,35,26,3,4,0,0,536.0,Sun,2019,2019-05-05


In [60]:
weather_import.head()

,dt,dt_iso,Year,Month,Day,Date,Hour,timezone,city_name,lat,...,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,Full_Date
0,946684800,2000-01-01 00:00:00 +0000 UTC,2000,1,1,2000-01-01,0,-18000,Acadia National Park Pond,44.378185,...,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n,2000-01-01
1,946688400,2000-01-01 01:00:00 +0000 UTC,2000,1,1,2000-01-01,1,-18000,Acadia National Park Pond,44.378185,...,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n,2000-01-01
2,946692000,2000-01-01 02:00:00 +0000 UTC,2000,1,1,2000-01-01,2,-18000,Acadia National Park Pond,44.378185,...,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n,2000-01-01
3,946695600,2000-01-01 03:00:00 +0000 UTC,2000,1,1,2000-01-01,3,-18000,Acadia National Park Pond,44.378185,...,NaN,NaN,NaN,NaN,40,802,Clouds,scattered clouds,03n,2000-01-01
4,946699200,2000-01-01 04:00:00 +0000 UTC,2000,1,1,2000-01-01,4,-18000,Acadia National Park Pond,44.378185,...,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n,2000-01-01


In [46]:
hourly_transformed_df = data_import.drop(columns = ["SrNo", "Month", "Day", "Year", "Total", "DOW"]).melt(id_vars=["Full_Date"],
                                                                                                          var_name="Time",
                                                                                                          value_name="Value").sort_values(by=['Full_Date', 'Time'])
hourly_transformed_df["Time"].loc[hourly_transformed_df['Time']=='24:00:00'] = '00:00'
hourly_transformed_df['Hour'] = pd.to_datetime(hourly_transformed_df['Time'], format='%H:%M').dt.hour
hourly_transformed_df.sort_values(by=['Full_Date', 'Hour'], inplace=True)

/Library/anaconda3/envs/ds_env/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [47]:
hourly_transformed_df.head()

,Full_Date,Time,Value,Hour
10580,2019-05-01,00:00,0,0
0,2019-05-01,1:00,0,1
460,2019-05-01,2:00,1,2
920,2019-05-01,3:00,0,3
1380,2019-05-01,4:00,0,4


In [61]:
hourly_traffic_weather = pd.merge(hourly_transformed_df, weather_import[['Full_Date', 'Hour', 'temp', 'visibility', 'dew_point', 'feels_like', 'temp_min', 'temp_max',                                                                            'pressure', 'humidity', 'wind_speed', 'wind_deg', 'wind_gust', 'rain_1h', 'rain_3h', 'snow_1h',                                                                           'snow_3h', 'weather_main', 'weather_description']],
                                  on=['Full_Date','Hour'])

In [62]:
hourly_traffic_weather.to_csv('All_Years_Weather_Merged.csv')